In [34]:
# import necessary libraries
import pandas as pd
import mysql.connector
import numpy as np


In [35]:
#MySQL database connection configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password':'s@dhikali2006'
} 

In [36]:
#establish connection to mysql database
conn = mysql.connector.connect(**db_config)

In [37]:
#function to execute mysl queries
def execute_query(query):
    cursor = conn.cursor()
    cursor.execute(query)
    conn.commit()
    cursor.close()

In [38]:
# create database if not exists
create_db_query = "CREATE DATABASE IF NOT EXISTS etl_example;"
execute_query(create_db_query)

In [39]:
#use the database
use_db_query = "USE etl_example;"
execute_query(use_db_query)

In [40]:
# create raw_data table
create_raw_data_table_query = """
CREATE TABLE IF NOT EXISTS raw_data (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    age INT,
    city VARCHAR(255)
);
"""
execute_query(create_raw_data_table_query)


In [41]:
# Generate sample data with null values and inconsistencies
num_rows = 100
names = ['John', 'Jane', 'Mike', 'Chris', None]  # including null values
ages = np.random.randint(18, 60, size=num_rows).astype(float)  # convert to float to allow null values
nan_indices = np.random.choice(range(num_rows), size=num_rows // 10, replace=False)  # select 10% indices for null values
ages[nan_indices] = np.nan  # introduce some null values

cities = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', ''] 
cities = cities * (num_rows // len(cities) + 1)  # extend the list to match the num_rows
np.random.shuffle(cities)  # shuffle to introduce inconsistencies
cities = cities[:num_rows]  # trim to match num_rows



In [45]:
#create a dataframe for the created sample data
sample_df = pd.DataFrame({
    'name': np.random.choice(names, size=num_rows),
    'age': ages,
    'city': cities
})

In [49]:
#load the dataframe into the database
#insert sample data into the raw_data table
for index, row in sample_df.iterrows():
    name_value = f"'{row['name']}'" if row['name'] is not  None else 'NULL'
    age_value = str(row['age']) if not np.isnan(row['age']) else 'NULL'
    city_value = f"'{row['city']}'" if row['city'] != '' else 'NULL'

    insert_query = f"""
    INSERT INTO raw_data (name, age, city)
    VALUES ({name_value}, {age_value}, {city_value}) 
    """

    execute_query(insert_query)


In [50]:
#fetch the data from the raw_data table and display
fetch_query = "SELECT * FROM raw_data"
fetched_df = pd.read_sql_query(fetch_query, conn)

fetched_df.head()


C:\Users\user\AppData\Local\Temp\ipykernel_6460\1300683070.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fetched_df = pd.read_sql_query(fetch_query, conn)


,id,name,age,city
0,1,Mike,57.0,Chicago
1,2,None,51.0,Los Angeles
2,3,None,35.0,None
3,4,Jane,28.0,Houston
4,5,Mike,57.0,Phoenix


In [67]:
## Transform the data

 # removing rows with missing values
cleaned_df = fetched_df.dropna()

# Remove rows with empty strings in 'city' column
cleaned_df = cleaned_df[cleaned_fetched_df['city'] != '']
                                        
# Convert 'age' column to integer type
cleaned_df['age'] = cleaned_df['age'].astype(int)

# Apply transformations to 'name' column
cleaned_df['name'] = cleaned_df['name'].str.upper()

cleaned_df.head()

,id,name,age,city
0,1,MIKE,57,Chicago
3,4,JANE,28,Houston
4,5,MIKE,57,Phoenix
7,8,MIKE,22,Chicago
8,9,CHRIS,24,Chicago


In [68]:
# create another database for the cleaned data
create_cleaned_table_query = """
CREATE TABLE IF NOT EXISTS cleaned_data (
     id INT AUTO_INCREMENT PRIMARY KEY,
     name VARCHAR(255),
     age INT,
     city VARCHAR(255)
);
"""
execute_query(create_cleaned_table_query)


In [71]:
#load the cleaned data to the new database
#insert cleaned data into mysql table
for index, row in cleaned_df.iterrows():
    insert_query = f"""
    INSERT INTO cleaned_data (name, age, city)
    VALUES ('{row['name']}', {row['age']}, '{row['city']}' )
    """

    execute_query(insert_query)

In [72]:
# view the cleaned data from the database
fetched_cleaned_query = "SELECT * FROM cleaned_data"
cleaned_fetched = pd.read_sql_query(fetched_cleaned_query, conn)
print(cleaned_fetched .head())

   id   name  age     city
0   1   MIKE   57  Chicago
1   2   JANE   28  Houston
2   3   MIKE   57  Phoenix
3   4   MIKE   22  Chicago
4   5  CHRIS   24  Chicago


C:\Users\user\AppData\Local\Temp\ipykernel_6460\1338218726.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cleaned_fetched = pd.read_sql_query(fetched_cleaned_query, conn)
